In [ ]:
# #Importing vol indices
# #source https://www.cboe.com/tradable_products/vix/vix_historical_data/
# vix = pd.read_csv('https://cdn.cboe.com/api/global/us_indices/daily_prices/VIX_History.csv')
# amz_vix = pd.read_csv('https://cdn.cboe.com/api/global/us_indices/daily_prices/VXAZN_History.csv')
# appl_vix = pd.read_csv('https://cdn.cboe.com/api/global/us_indices/daily_prices/VXAPL_History.csv')


In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from polygon import RESTClient
import datetime as dt
import os
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import os
import shutil
import requests
import aiohttp
import asyncio
from datetime import datetime, timedelta

In [2]:
load_dotenv()
api_key = os.getenv("API_KEY")
client = RESTClient(api_key)


In [4]:
wiki = 'http://en.wikipedia.org/wiki'
djia_ticker_list = wiki + '/Dow_Jones_Industrial_Average'
sp500_tickers_list = wiki + '/List_of_S%26P_500_companies'
tickersSP500 = pd.read_html(sp500_tickers_list)[0].Symbol.to_list()
djia_tickers = pd.read_html(djia_ticker_list)[1].Symbol.to_list()

In [8]:
class DataPipeline:
    def __init__(self, client, tickers, start_date = '2003-09-09', end_date = '2024-03-28', timespan = 'minute', multiplier = 1, limit = 50000):
        self.client = client
        self.tickers = tickers
        self.start_date = start_date
        self.end_date = end_date
        self.timespan = timespan
        self.multiplier = multiplier
        self.limit = limit
    
    def fetch_data(self, ticker):
            data= pd.DataFrame(self.client.list_aggs(ticker,timespan = self.timespan, multiplier = self.multiplier,from_ = self.start_date, to = self.end_date,limit=self.limit))
            data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms').dt.tz_localize('UTC').dt.tz_convert('US/Eastern').dt.tz_localize(None)
            data = data.rename(columns = {'timestamp': 'Date'}).set_index('Date')
            filtered_data = data.between_time('09:30', '16:05')
            filtered_data = filtered_data.drop(columns = ['open', 'vwap', 'high', 'low', 'volume', 'transactions', 'otc'])
            data = data.drop(columns = ['open', 'vwap', 'high', 'low', 'volume', 'transactions', 'otc'])
            return data, filtered_data
        
    

            
    def process_data(self,df):
        df['Day'] = pd.to_datetime(df.index).normalize()
        df['log_ret'] = np.log(df['close']).diff()
        df['squared_log_ret'] = df['log_ret']**2
        df['returns'] = df['close'].pct_change()
        df['squared_ret'] = df['returns']**2
        return df
    
    def calculate_realized_var(self,df):
        realized_var = df.groupby('Day').agg({'squared_log_ret': 'sum'})
        return realized_var.rename(columns ={'squared_log_ret': 'daily_realised_log_var'})

    
    def processing_pipeline(self):
        results = {}
        for ticker in self.tickers:
            data, filtered_data = self.fetch_data(ticker)
            processed_data = self.process_data(data)
            processed_filtered_data = self.process_data(filtered_data)
            
            realized_var = self.calculate_realized_var(processed_data)
            realized_var_filtered = self.calculate_realized_var(processed_filtered_data)
            
            combined_data = realized_var.join(realized_var_filtered, lsuffix='', rsuffix='_filtered')
            
            
            combined_data.to_csv(f'{ticker}_realized_combined.csv')
            processed_data.to_csv(f'{ticker}_data.csv')
            processed_filtered_data.to_csv(f'{ticker}_filtered_data.csv')
           
            results[ticker] = combined_data
            

        return results
        


In [9]:
update_data = DataPipeline(client=client, tickers= ['QQQ'])

In [10]:
result = update_data.processing_pipeline()

In [7]:
pd.read_csv('/Users/raphaelravinet/Code/BSE/Thesis/QQQ_filtered_data.csv')

,Date,close,Day,log_ret,squared_log_ret,returns,squared_ret
0,2003-09-10 09:30:00,33.7500,2003-09-10,NaN,NaN,NaN,NaN
1,2003-09-10 09:35:00,33.8300,2003-09-10,0.002368,5.605366e-06,0.002370,5.618656e-06
2,2003-09-10 09:40:00,33.7400,2003-09-10,-0.002664,7.096393e-06,-0.002660,7.077519e-06
3,2003-09-10 09:45:00,33.7200,2003-09-10,-0.000593,3.515826e-07,-0.000593,3.513742e-07
4,2003-09-10 09:50:00,33.8200,2003-09-10,0.002961,8.768767e-06,0.002966,8.794778e-06
...,...,...,...,...,...,...,...
286049,2024-03-28 15:45:00,444.4895,2024-03-28,-0.000460,2.115748e-07,-0.000460,2.114775e-07
286050,2024-03-28 15:50:00,444.4250,2024-03-28,-0.000145,2.106005e-08,-0.000145,2.105700e-08
286051,2024-03-28 15:55:00,443.9700,2024-03-28,-0.001024,1.049230e-06,-0.001024,1.048156e-06
286052,2024-03-28 16:00:00,443.9400,2024-03-28,-0.000068,4.566302e-09,-0.000068,4.565993e-09


In [19]:
# #Moving FILES
# directory = '/Users/raphaelravinet/Code/BSE/Thesis'

# # Target directory 
# directory2 = '/Users/raphaelravinet/Code/BSE/Thesis/data'

# files = [f for f in os.listdir(directory) if f.endswith('.csv')]


# tickers = set(f.split('_')[0] for f in files)


# for ticker in tickers:

#     ticker_folder = os.path.join(directory2, ticker)
    
#     # Create folder if it doesn't exist
#     if not os.path.exists(ticker_folder):
#         os.makedirs(ticker_folder)
    
#     for file in files:
#         if file.startswith(ticker):
#             shutil.move(os.path.join(directory, file), os.path.join(ticker_folder, file))

# print("Folders created and files moved successfully.")


Folders created and files moved successfully.


In [15]:
# def calculate_returns(df):
#     df['returns'] = df['close'].pct_change()
#     return df


# def process_folder(base_dir):
#     for root, dirs, files in os.walk(base_dir):
#         for file in files:
#             if file.endswith("_filtered_data.csv") or file.endswith("_daily_data.csv"):
#                 file_path = os.path.join(root, file)
#                 df = pd.read_csv(file_path)
#                 df = calculate_returns(df)
#                 df.to_csv(file_path, index=False)


# # Example usage:
# base_dir = "/Users/raphaelravinet/Code/BSE/Thesis/data"
# process_folder(base_dir)


In [7]:
# Replace this with your actual client fetching method
def fetch_data(client, ticker, start_date, end_date, timespan, multiplier, limit):
    data = pd.DataFrame(client.list_aggs(ticker, timespan=timespan, multiplier=multiplier, from_=start_date, to=end_date, limit=limit))
    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms').dt.tz_localize('UTC').dt.tz_convert('US/Eastern').dt.tz_localize(None)
    data = data.rename(columns={'timestamp': 'Date'}).set_index('Date')
    data = data.drop(columns=['open', 'vwap', 'high', 'low', 'volume', 'transactions', 'otc'])
    return data

def process_data(df):
    df['Day'] = pd.to_datetime(df.index).normalize()
    df['log_ret'] = np.log(df['close']).diff()
    df['returns'] = df['close'].pct_change()
    return df

# Assuming `client` is already instantiated and available
tickers = ['SPY', 'QQQ']
start_date = '2003-09-10'
end_date = '2024-03-28'
timespan = 'day'
multiplier = 1
limit = 50000

for ticker in tickers:
    data = fetch_data(client, ticker, start_date, end_date, timespan, multiplier, limit)
    processed_data = process_data(data)
    processed_data.to_csv(f'{ticker}_daily_data.csv', index=False)
    print(f"Processed data for {ticker} saved to {ticker}_daily_data.csv")

# Check the first few rows of the processed data
spy_data = pd.read_csv('SPY_daily_data.csv')
qqq_data = pd.read_csv('QQQ_daily_data.csv')

print(spy_data.head())
print(qqq_data.head())


Processed data for SPY saved to SPY_daily_data.csv
Processed data for QQQ saved to QQQ_daily_data.csv
    close         Day   log_ret   returns
0  101.96  2003-09-10       NaN       NaN
1  102.26  2003-09-11  0.002938  0.002942
2  102.45  2003-09-12  0.001856  0.001858
3  102.09  2003-09-15 -0.003520 -0.003514
4  103.58  2003-09-16  0.014489  0.014595
   close         Day   log_ret   returns
0  33.27  2003-09-10       NaN       NaN
1  33.64  2003-09-11  0.011060  0.011121
2  33.82  2003-09-12  0.005337  0.005351
3  33.49  2003-09-15 -0.009805 -0.009758
4  34.40  2003-09-16  0.026810  0.027172
